In [2]:
import pandas as pd
from raw_fetch import download_play_by_play, download_players, download_snap_counts, download_player_stats

In [6]:
download_players()

In [7]:
snap_count = pd.read_parquet('raw/snap_counts/snap_counts_2024.parquet')

In [8]:
players = pd.read_parquet('raw/players/players.parquet')

In [11]:
stats = pd.read_parquet('raw/player_stats/player_stats_2024.parquet')

In [28]:
# player with the highest wopr
qb_stats = stats[stats['position'] == 'QB'][[
    'week',
    'player_name',
    'attempts',
    'passing_epa',
    'completions',
    'passing_yards',
    'passing_air_yards',
    'passing_first_downs',
    'rushing_yards',
    'passing_yards_after_catch']]
qb_stats['rank'] = qb_stats['attempts'].rank(method='min', ascending=False)
qb_stats['completion_pct'] = qb_stats['completions'] / qb_stats['attempts']
qb_stats['yards_after_catch'] = qb_stats['passing_yards'] - \
    qb_stats['passing_air_yards']
# calculate week over week change of first downs
qb_stats['first_down_change'] = qb_stats.groupby(
    'player_name')['passing_first_downs'].diff()
qb_stats_sorted = qb_stats.sort_values('rushing_yards', ascending=False)
qb_stats_sorted[qb_stats_sorted['week'] == 4]

,week,player_name,attempts,passing_epa,completions,passing_yards,passing_air_yards,passing_first_downs,rushing_yards,passing_yards_after_catch,rank,completion_pct,yards_after_catch,first_down_change
677,4,J.Fields,34,3.554026,22,312.0,240.0,11.0,55.0,156.0,35.0,0.647059,72.0,2.0
331,4,L.Jackson,18,9.182409,13,156.0,115.0,9.0,54.0,92.0,121.0,0.722222,41.0,1.0
1216,4,J.Daniels,30,13.108258,26,233.0,174.0,14.0,47.0,122.0,57.0,0.866667,59.0,1.0
484,4,T.Huntley,22,-10.815693,14,96.0,176.0,4.0,40.0,55.0,108.0,0.636364,-80.0,NaN
45,4,G.Smith,56,9.814202,38,395.0,305.0,22.0,38.0,214.0,1.0,0.678571,90.0,9.0
187,4,D.Watson,32,-6.922351,24,176.0,172.0,9.0,32.0,102.0,48.0,0.750000,4.0,2.0
3,4,A.Rodgers,42,-12.087542,24,225.0,306.0,10.0,26.0,96.0,10.0,0.571429,-81.0,-6.0
169,4,T.Hill,0,NaN,0,0.0,0.0,0.0,24.0,0.0,146.0,NaN,0.0,0.0
1094,4,A.Richardson,4,4.277929,3,71.0,56.0,3.0,24.0,23.0,136.0,0.750000,15.0,-1.0
363,4,J.Allen,29,-7.575485,16,180.0,320.0,6.0,21.0,63.0,66.0,0.551724,-140.0,-9.0


In [19]:
qb_stats_sorted[qb_stats_sorted['player_name'].str.contains('S.Dar')]

,week,player_name,attempts,passing_epa,completions,passing_yards,passing_air_yards,passing_first_downs,passing_yards_after_catch,rank,completion_pct,yards_after_catch,first_down_change
364,1,S.Darnold,24,9.554716,19,208.0,195.0,8.0,68.0,98.0,0.791667,13.0,NaN
365,2,S.Darnold,26,-1.314427,17,268.0,169.0,9.0,152.0,87.0,0.653846,99.0,1.0
366,3,S.Darnold,28,5.252612,17,181.0,191.0,12.0,67.0,75.0,0.607143,-10.0,3.0
367,4,S.Darnold,28,8.126838,20,275.0,289.0,11.0,115.0,75.0,0.714286,-14.0,-1.0


In [13]:
stats.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [16]:
snap_count.columns

Index(['game_id', 'pfr_game_id', 'season', 'game_type', 'week', 'player',
       'pfr_player_id', 'position', 'team', 'opponent', 'offense_snaps',
       'offense_pct', 'defense_snaps', 'defense_pct', 'st_snaps', 'st_pct'],
      dtype='object')

In [32]:
# player with the highest wopr
rb_stats = stats[stats['position'] == 'RB'][[
    'week',
    'player_name',
    'carries',
    'rushing_epa',
    'carries',
    'rushing_yards',
    'rushing_tds',
    'rushing_fumbles',
    'rushing_fumbles_lost',
    'rushing_first_downs',
    'rushing_epa',
    'rushing_2pt_conversions',
    'receptions',
    'targets'
]]
# rb_stats['rank'] = rb_stats['carries'].rank(method='min', ascending=False)
rb_stats['ypc'] = rb_stats['rushing_yards'] / rb_stats['carries']
# calculate week over week change of first downs
rb_stats['first_down_change'] = rb_stats.groupby(
    'player_name')['rushing_first_downs'].diff()
rb_stats_sorted = rb_stats.sort_values('rushing_yards', ascending=False)
rb_stats_sorted[qb_stats_sorted['week'] == 4]

ValueError: cannot reindex on an axis with duplicate labels